In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from random import shuffle
from random import randint
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from operator import itemgetter
import time
import xgboost as xgb
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

import gc

In [0]:
dfxd = pd.read_csv('/content/drive/My Drive/zmienne_final.csv').drop(columns=['Unnamed: 0'])
pd.set_option('display.max_columns', None)

print(dfxd.shape)
dfxd.head()

(144233, 59)


,isFraud,id_01,id_02,id_05,id_06,id_13,id_12,id_19,id_20,id_30,id_31,ProductCD,DeviceType,id_33,DeviceInfo,TransactionAmt,card1,card2,card3,card4,card5,card6,addr1,dist2,P_emaildomain,R_emaildomain,C1,C2,C4,C6,C7,C8,C10,C11,C12,C13,C14,D2,D6,D8,D9,V44,V45,V86,V87,V149,V156,V188,V189,V200,V201,V203,V210,V245,V257,V258,V259,V264,V265
0,0,0.0,70787.0,-999.0,-999.0,-999.0,NotFound,542.0,144.0,Android,samsung,H,mobile,2000<x<2500,SAMSUNG,50.000,4497,514.0,150.0,mastercard,102.0,credit,420.0,-999.0,gmail,-999,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,1.0,1.0,0.000000,0.000000
1,0,-5.0,98945.0,0.0,-5.0,49.0,NotFound,621.0,500.0,IOS,safari,H,mobile,1000<x<1500,iOS,15.000,2803,100.0,150.0,visa,226.0,debit,337.0,-999.0,anonymous,-999,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,1.0,1.0,0.000000,0.000000
2,0,-5.0,191631.0,0.0,0.0,52.0,NotFound,410.0,142.0,-999,chrome,C,desktop,-999,WINDOWS,75.887,16496,352.0,117.0,mastercard,134.0,credit,-999.0,-999.0,gmail,gmail,1.0,4.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,0.0,83.0,0.0,1.0,1.0,1.0,1.0,-999.0,-999.0,1.0,1.0,1.0,1.0,166.215393,90.327904,1.0,1.0,1.0,1.0,166.215393,166.215393
3,0,-5.0,221832.0,0.0,-6.0,52.0,NotFound,176.0,507.0,-999,chrome,C,desktop,-999,-999,16.495,4461,375.0,185.0,mastercard,224.0,debit,-999.0,30.0,hotmail,hotmail,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-999.0,0.0,-999.0,-999.0,1.0,1.0,1.0,1.0,-999.0,-999.0,1.0,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,1.0,1.0,0.000000,0.000000
4,0,0.0,7460.0,1.0,0.0,-999.0,NotFound,529.0,575.0,Mac,chrome,H,desktop,1000<x<1500,INNE,30.000,1790,555.0,150.0,visa,226.0,debit,170.0,-999.0,aol,-999,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,-999.0,-999.0,26.0,0.0,-999.0,-999.0,-999.0,-999.0,0.0,0.0,1.0,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0,1.0,1.0,0.000000,0.000000


In [0]:
target = "isFraud"
catFeatures=['id_30', 'id_31', 'id_33', 'P_emaildomain', 'R_emaildomain', 'ProductCD', 'DeviceType', 'id_12', 'DeviceInfo', 'card4','card6']

numFeatures=list(dfxd)
for x in catFeatures:
  numFeatures.remove(x)

numFeatures.remove('isFraud')
for x in catFeatures:
  dfxd[x] = dfxd[x].astype('str')

In [0]:
df = dfxd.copy()
mapy={}
for feature in catFeatures:
    le = preprocessing.LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    mapy[feature] = le

In [0]:
features = df.columns.tolist()
features.remove('isFraud')
print(features)

['id_01', 'id_02', 'id_05', 'id_06', 'id_13', 'id_12', 'id_19', 'id_20', 'id_30', 'id_31', 'ProductCD', 'DeviceType', 'id_33', 'DeviceInfo', 'TransactionAmt', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C4', 'C6', 'C7', 'C8', 'C10', 'C11', 'C12', 'C13', 'C14', 'D2', 'D6', 'D8', 'D9', 'V44', 'V45', 'V86', 'V87', 'V149', 'V156', 'V188', 'V189', 'V200', 'V201', 'V203', 'V210', 'V245', 'V257', 'V258', 'V259', 'V264', 'V265']


In [0]:
target='isFraud'

In [0]:
df['isFraud'] = df['isFraud'].astype(int)

In [0]:
def run_xgb(train, validate, features, target, 
            eta=0.03, max_depth=7, subsample = 0.7, colsample_bytree = 0.7, # hiperparametry
            colsample_bylevel=1,lambdaX = 1, alpha=0, gamma=0, min_child_weight=0, # hiperparametry
            rate_drop = 0.2, skip_drop=0.5, # hiperparametry
            num_boost_round = 1000, early_stopping_rounds = 50, # hiperparametry
            debug=True, eval_metric= ["aucpr"], objective = "binary:logistic", # konfiguracja
            seed=2020, booster = "gbtree", tree_method="exact", grow_policy="depthwise"): # konfiguracja
  
   
    start_time = time.time()
    paramList= ['eta', 'max_depth',
         'subsample', 'colsample_bytree', 'colsample_bylevel',
         'lambdaX', 'alpha', 'gamma', 'min_child_weight',
         'num_boost_round', 'early_stopping_rounds',
         'rate_drop', 'skip_drop',
         'eval_metric', 'objective', 
          'seed', 'booster', 'tree_method', 'grow_policy']
    
    params = dict()
    for param in paramList:
        params[param]=eval(param)
    if(debug):
        for param in paramList:
            print(param, eval(param), end=", ")
        print('\nLength train:', len(train.index))
        print('Length valid:', len(validate.index))

    if (params["objective"]=="multi:softmax" or params["objective"]=="multi:softprob"):
        params["num_class"]=train[target].nunique()
    params["silent"]=1
    
    params["lambda"]=lambdaX
    
    dtrain = xgb.DMatrix(train[features].values, train[target].values, feature_names=train[features].columns.values)
    dvalid = xgb.DMatrix(validate[features].values, validate[target].values, feature_names=validate[features].columns.values)

    evals = [(dtrain, 'train'), (dvalid, 'valid')]
    
    train_history = dict()

    gbm = xgb.train(params, dtrain, 
                    num_boost_round, early_stopping_rounds=early_stopping_rounds,
                    evals=evals, evals_result=train_history, verbose_eval=debug)
    
    score = gbm.best_score

    trainHistory = dict()
    for key in train_history.keys():
        for metric in train_history[key].keys():
            trainHistory[key+metric.upper()]=train_history[key][metric]
    train_history = pd.DataFrame(trainHistory)

    trainPred = gbm.predict(dtrain, ntree_limit=gbm.best_iteration)
    testPred = gbm.predict(dvalid, ntree_limit=gbm.best_iteration)

    imp = gbm.get_fscore()
    imp = sorted(imp.items(), key=itemgetter(1), reverse=True)

    impFig=None
    if(debug):
        print('Czas trenowania: {} minut'.format(round((time.time() - start_time)/60, 2)))
        impFig, ax = plt.subplots()
        xgb.plot_importance(gbm, ax=ax)
    return score, trainPred, testPred, train_history, impFig, imp

In [0]:
resHists = dict()
rang = range(4,15,2)
for x in rang:
    score, trainPred, testPred, train_history, impFig, imp = run_xgb(X_train, X_valid, features, target,
                                                          max_depth=x, early_stopping_rounds=50, debug=False)
    resHists[x]=train_history
    print(x, score)
fig, ax = plt.subplots(1, 2, figsize=(14,6))
for x in rang:
    resHists[x][['trainAUC']].add_suffix('_'+str(x)).iloc[10:].plot(ax=ax[0])
    resHists[x][['validAUC']].add_suffix('_'+str(x)).iloc[10:].plot(ax=ax[1])
plt.show()

In [0]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

def CVTestXGB(nFolds = 5, randomState=2020, features=features, debug=False, *args, **kwargs):
    
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)


    testResults = []
    trainResults = []
    
    predictions = []
    

    indices = []
    
   
    hists = []
    fold = 1
    for train, test in kf.split(df.index.values):
        
        score, trainPred, testPred, train_history, impFig, imp = run_xgb(df.iloc[train], df.iloc[test],
                                                                         features, target, debug=debug, *args, **kwargs)
        
     
        predictions.append(testPred.tolist().copy())
        
        indices.append(df.iloc[test].index.tolist().copy())
        
        trainScore = average_precision_score(df[target].iloc[train], trainPred)
        testScore = average_precision_score(df[target].iloc[test], testPred)
        
        trainResults.append(trainScore)
        testResults.append(testScore)
        
        hists.append(train_history.add_suffix('_'+str(fold)))
        fold+=1
        
        if debug:
            print("Train AUC:", trainScore,
                  "Valid AUC:", testScore)
        
    return trainResults, testResults, predictions, indices, pd.concat(hists, axis=1)

In [0]:
score, trainPred, testPred, train_history, impFig, imp = run_xgb(debug=True)

In [0]:
trainResults, testResults, predictions, indices, hists = CVTestXGB(
    max_depth=8, lambdaX=4, subsample=0.7, colsample_bytree=0.7, gamma=0.1, debug=False)
np.mean(trainResults), np.mean(testResults)

In [0]:
import xgboost as xgb


In [0]:
for k in [12]:
  trainResults, testResults, predictions, indices, hists = CVTestXGB(max_depth=k)
  print(k, np.mean(trainResults), np.mean(testResults))

In [0]:
import random
results=[]
paramList = []
for x in range(100):
    params = (random.randint(3, 20), random.uniform(0.1, 0.9), random.uniform(0.1, 0.9), random.uniform(0, 10), random.uniform(0, 10))
    trainResults, testResults, predictions, indices, hists = CVTestXGB(
        max_depth=params[0],
        subsample = params[1],
        colsample_bytree = params[2],
        lambdaX=params[3],
        gamma=params[4])
    print(np.mean(testResults), params)
    results.append(np.mean(testResults))
    paramList.append(params)


0.8977003204558296 (14, 0.6488265035352965, 0.2943797831513056, 3.3774568938557348, 3.9730743309603342)
0.887780869464023 (14, 0.21820739124388722, 0.27759420819987857, 1.4258813046727237, 3.21755952941274)
0.8911942548166458 (20, 0.4259214761504847, 0.154877353431075, 0.8320627580993623, 4.422722234450891)
0.8073568721207331 (3, 0.22617538529715056, 0.24136266021484076, 0.8094052254319606, 0.33266654834043674)
0.823479798442043 (4, 0.47283840061110627, 0.6796195791659454, 5.357244442483795, 9.433353780231021)
0.8963268719076665 (10, 0.19438065052238374, 0.37773920600257027, 3.694904619452044, 0.4304566235008134)
0.8133730470442444 (5, 0.10667508958562077, 0.8524271399417728, 8.165151325446232, 5.575534246370309)
0.8662613878456165 (16, 0.384956323980725, 0.3701320257930759, 9.106213142508626, 4.776157070985007)
0.8582846192625027 (7, 0.5741966148325718, 0.7647994632508665, 4.6244462428029856, 8.698726675627302)
0.9160501325931707 (16, 0.4904793337160137, 0.7316860537366651, 5.55601245

In [0]:
resHists = dict()
rang = range(3,10)
for x in rang:
    score, trainPred, testPred, train_history, impFig, imp = run_xgb(X_train, X_valid, features, target,
                                                          max_depth=8,  subsample=x/10,
                                                          early_stopping_rounds=50, debug=False)
    resHists[x]=train_history
    print(x/10, score)
fig, ax = plt.subplots(1, 2, figsize=(14,6))
for x in rang:
    resHists[x][['trainAUC']].add_suffix('_'+str(x)).iloc[10:].plot(ax=ax[0])
    resHists[x][['validAUC']].add_suffix('_'+str(x)).iloc[10:].plot(ax=ax[1])
plt.show()

In [0]:
trainResults, testResults, predictions, indices, hists = CVTestXGB(
        max_depth=16, subsample = 0.4904793337160137, colsample_bytree = 0.7316860537366651, lambdaX=5.5560124578661485, gamma=0.15318592498024586)
print(np.mean(testResults))

modelXGB = {
    "name":"XGB",
    "description":"Model XGB, ze zmiennymi kategorycznymi z LE",
    "specification":'max_depth=7, subsample = 0.8, colsample_bytree = 0.7, lambdaX=1, gamma=6',
    "trainResults":trainResults.copy(),
    "testResults":testResults.copy(),
    "predictions":predictions.copy(),
    "indices":indices.copy(),
}

0.9160501325931707


In [0]:
import pickle

In [0]:
with open("model_XGB_1.p", "wb") as fp:
    pickle.dump(modelXGB, fp)